<a href="https://colab.research.google.com/github/xhxuciedu/cs273p/blob/main/pytorch-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
import numpy as np
import matplotlib.pylab as plt
import sys
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

In [2]:
from IPython.display import HTML, display


def set_default(figsize=(8, 5), dpi=100):
    plt.style.use(["dark_background", "bmh"])
    plt.rc("axes", facecolor="k")
    plt.rc("figure", facecolor="k")
    plt.rc("figure", figsize=figsize, dpi=dpi)


set_default()

### Check Package Versions

In [3]:
print("__Python VERSION:", sys.version)
print("__PyTorch VERSION:", torch.__version__)
print("__CUDNN VERSION:", torch.backends.cudnn.version())
print("__Number CUDA Devices:", torch.cuda.device_count())

__Python VERSION: 3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ]
__PyTorch VERSION: 2.1.2
__CUDNN VERSION: None
__Number CUDA Devices: 0


### PyTorch

What is PyTorch?

It’s a Python based scientific computing package targeted at two sets of audiences:

- A replacement for numpy to use the power of GPUs
- a deep learning research platform that provides maximum flexibility and speed


### Tensors

Tensors are similar to numpy’s ndarrays, with the addition being that Tensors can also be used on a GPU to accelerate computing.

Construct a 5x3 matrix, uninitialized


In [4]:
x = torch.Tensor(5, 3)
print(x)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [5]:
# get its size
y = torch.rand(5, 3)
print(x + y)

tensor([[0.8874, 0.2617, 0.4787],
        [0.8266, 0.1494, 0.5140],
        [0.0977, 0.4356, 0.1170],
        [0.7623, 0.3481, 0.8324],
        [0.4512, 0.9530, 0.6559]])


In [6]:
# Addition: in-place
y.add_(x)

tensor([[0.8874, 0.2617, 0.4787],
        [0.8266, 0.1494, 0.5140],
        [0.0977, 0.4356, 0.1170],
        [0.7623, 0.3481, 0.8324],
        [0.4512, 0.9530, 0.6559]])

### Create tensors

In [7]:
# random
v = torch.rand(2, 3)  # Initialize with random number (uniform distribution)
v = torch.randn(2, 3)  # With normal distribution (SD=1, mean=0)
v = torch.randperm(4)

# ones
eye = torch.eye(3)  # Create an identity 3x3 tensor
v = torch.ones(10)  # A tensor of size 10 containing all ones
v = torch.ones(2, 1, 2, 1)  # Size 2x1x2x1
v = torch.ones_like(eye)  # A tensor with same shape as eye. Fill it with 1.

# zeros
v = torch.zeros(10)

# range of values
v = torch.arange(5)  # similar to range(5) but creating a Tensor
v = torch.arange(0, 5, step=1)  # Size 5. Similar to range(0, 5, 1)

# linear or log scale
v = torch.linspace(
    1, 10, steps=10
)  # Create a Tensor with 10 linear points for (1, 10) inclusively
v = torch.logspace(
    start=-10, end=10, steps=5
)  # Size 5: 1.0e-10 1.0e-05 1.0e+00, 1.0e+05, 1.0e+10

### Dot product, component-wide product, matrix multiplication, 

In [8]:
# Dot product of 2 tensors
r = torch.dot(torch.Tensor([4, 2]), torch.Tensor([3, 1]))  # 14

In [9]:
# component-wise product
torch.Tensor([4, 2]) * torch.Tensor([3, 1])

tensor([12.,  2.])

In [10]:
# Matrix x Matrix
# Size 2x4
mat1 = torch.randn(2, 3)
mat2 = torch.randn(3, 4)
r = torch.mm(mat1, mat2)

In [11]:
# Batch Matrix x Matrix
# Size 10x3x5
batch1 = torch.randn(10, 3, 4)
batch2 = torch.randn(10, 4, 5)
r = torch.bmm(batch1, batch2)

###Squeeze and unsqueeze

In [12]:
t = torch.ones(2, 1, 2, 1)  # Size 2x1x2x1
r = torch.squeeze(t)  # Size 2x2
r = torch.squeeze(t, 1)  # Squeeze dimension 1: Size 2x2x1

# Un-squeeze a dimension
x = torch.Tensor([1, 2, 3])
r = torch.unsqueeze(x, 0)  # Size: 1x3
r = torch.unsqueeze(x, 1)  # Size: 3x1

### Transpose


In [13]:
# Transpose dim 0 and 1
v = torch.randn(3, 2)
r = torch.transpose(v, 0, 1)
print(r.shape)

torch.Size([2, 3])


### Numpy Bridge

Converting a torch Tensor to a numpy array and vice versa is a breeze.

The torch Tensor and numpy array will share their underlying memory locations, and changing one will change the other.

Converting torch Tensor to numpy Array


In [14]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

In [15]:
# Conversion
a = np.array([1, 2, 3])
v = torch.from_numpy(a)  # Convert a numpy array to a Tensor

b = v.numpy()  # Tensor to numpy
b[1] = -1  # Numpy and Tensor share the same memory
assert a[1] == b[1]  # Change Numpy will also change the Tensor

### Reshape tensor

In [16]:
### Tensor resizing
x = torch.randn(2, 3)  # Size 2x3
y = x.view(6)  # Resize x to size 6
z = x.view(-1, 2)  # Size 3x2
print(y.shape, z.shape)

torch.Size([6]) torch.Size([3, 2])


###CUDA Tensors

All the Tensors on the CPU except a CharTensor support converting to NumPy and back.

Tensors can be moved onto GPU using the .cuda function.


In [17]:
# let us run this cell only if CUDA is available

x = torch.rand(3, 2)
y = torch.rand(3, 2)
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y

In [18]:
x

tensor([[0.9127, 0.4743],
        [0.5825, 0.0882],
        [0.1765, 0.9361]])

In [19]:
y

tensor([[0.3169, 0.9096],
        [0.3209, 0.7751],
        [0.3216, 0.9632]])

## Autograd: automatic differentiation

Central to all neural networks in PyTorch is autograd, a core torch package for automatic differentiation.

The autograd package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.


In [20]:
# create an variable
x = torch.ones((2, 2), requires_grad=True)

# Do an operation of variable:
y = x + 2

# Do more operations on y
z = y * y * 3
out = z.mean()

In [21]:
# Gradients
# ---------
# let's backprop now
# ``out.backward()`` is equivalent to doing ``out.backward(torch.Tensor([1.0]))``
out.backward()

In [22]:
###############################################################
# print gradients d(out)/dx
#

print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


You should have got a matrix of `4.5`. Let’s call the `out` _Variable_ $o$.
We have that: $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$

Therefore,
$$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$$ hence
$$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$$


In [23]:
# You can do many crazy things with autograd!
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ 821.7240, 1491.2695,  926.4733], grad_fn=<MulBackward0>)


In [24]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


### Basic autograd example 1 

In [25]:
# Create tensors.
x = torch.tensor(1.0, requires_grad=True)
w = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)

In [26]:
# Build a computational graph.
y = w * x + b  # y = 2 * x + 3

In [27]:
# Compute gradients.
y.backward()

In [28]:
# Print out the gradients.
print(x.grad)  # x.grad = 2
print(w.grad)  # w.grad = 1
print(b.grad)  # b.grad = 1

tensor(2.)
tensor(1.)
tensor(1.)


In [29]:
y.detach().numpy()

array(5., dtype=float32)

### Basic autograd example 2  

In [30]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)
print(x.shape, y.shape)

torch.Size([10, 3]) torch.Size([10, 2])


In [31]:
# Build a fully connected layer.
linear = nn.Linear(3, 2)
print("w: ", linear.weight)
print("b: ", linear.bias)

w:  Parameter containing:
tensor([[-0.0104, -0.0041, -0.3171],
        [ 0.1509, -0.4402, -0.3914]], requires_grad=True)
b:  Parameter containing:
tensor([0.4418, 0.0423], requires_grad=True)


In [32]:
loss = torch.sum((linear(x) - y) ** 2) / y.shape[0]
print("loss: ", loss.data.numpy())

loss:  1.7213104


In [33]:
loss.backward()

In [34]:
print("w grad: ", linear.weight.grad)
print("b grad: ", linear.bias.grad)

w grad:  tensor([[-0.3471,  0.0908, -0.3965],
        [-0.3288, -1.0883,  0.4767]])
b grad:  tensor([-0.0294,  0.7396])


In [35]:
# check grad
print("w grad:", (linear(x) - y).transpose(0, 1).mm(x) / y.shape[0] * 2)
print("b grad:", 2 * torch.mean(linear(x) - y, dim=0))

w grad: tensor([[-0.3471,  0.0908, -0.3965],
        [-0.3288, -1.0883,  0.4767]], grad_fn=<MulBackward0>)
b grad: tensor([-0.0294,  0.7396], grad_fn=<MulBackward0>)


###. Basic autograd example 3

In [36]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)
linear = nn.Linear(3, 2)

In [37]:
# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print("loss: ", loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print("dL/dw: ", linear.weight.grad)
print("dL/db: ", linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print("loss after 1 step optimization: ", loss.item())

loss:  1.570475459098816
dL/dw:  tensor([[-1.0785,  0.0653, -0.4341],
        [-0.3406,  0.2870, -0.3502]])
dL/db:  tensor([0.4043, 0.0447])
loss after 1 step optimization:  1.5522257089614868


## Input pipeline, Data loader

In [38]:
# # Download and construct CIFAR-10 dataset.
# train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
#                                              train=True,
#                                              transform=transforms.ToTensor(),
#                                              download=True)

# # Fetch one data pair (read data from disk).
# image, label = train_dataset[0]
# print (image.size())
# print (label)

# # Data loader (this provides queues and threads in a very simple way).
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                            batch_size=64,
#                                            shuffle=True)

# # When iteration starts, queue and thread start to load data from files.
# data_iter = iter(train_loader)

# # Mini-batch images and labels.
# images, labels = data_iter.next()

# # Actual usage of the data loader is as below.
# for images, labels in train_loader:
#     # Training code should be written here.
#     pass

### Input pipeline for custom dataset

In [39]:
# # ================================================================== #
# #                  Input pipeline for custom dataset                 #
# # ================================================================== #

# # You should build your custom dataset as below.
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self):
#         # TODO
#         # 1. Initialize file paths or a list of file names.
#         pass
#     def __getitem__(self, index):
#         # TODO
#         # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
#         # 2. Preprocess the data (e.g. torchvision.Transform).
#         # 3. Return a data pair (e.g. image and label).
#         pass
#     def __len__(self):
#         # You should change 0 to the total size of your dataset.
#         return 0

# # You can then use the prebuilt data loader.
# custom_dataset = CustomDataset()
# train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
#                                            batch_size=64,
#                                            shuffle=True)

### Pretrained model

In [40]:
# # ================================================================== #
# #                           Pretrained model                         #
# # ================================================================== #

# # Download and load the pretrained ResNet-18.
# resnet = torchvision.models.resnet18(pretrained=True)

# # If you want to finetune only the top layer of the model, set as below.
# for param in resnet.parameters():
#     param.requires_grad = False

# # Replace the top layer for finetuning.
# resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# # Forward pass.
# images = torch.randn(64, 3, 224, 224)
# outputs = resnet(images)
# print (outputs.size())     # (64, 100)

### Save and load the model    

In [41]:
# # Save and load the entire model.
# torch.save(resnet, 'model.ckpt')
# model = torch.load('model.ckpt')

# # Save and load only the model parameters (recommended).
# torch.save(resnet.state_dict(), 'params.ckpt')
# resnet.load_state_dict(torch.load('params.ckpt'))